<h1>Sentiment analysis</h1>

<h3>Import delle librerie e del dataset</h3>

In [ ]:
#import primari
import pandas as pd
import nltk
df = pd.read_csv("dataset_italianoEinglese-UPDATED_2023-6-16.csv", low_memory=False)
df.head(5)

In [ ]:
#Altri import necessari
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter

<p>Tokenizzazione del testo con creazione di colonna specifica</p>

In [ ]:
df['tokens'] = df['tweetText'].apply(nltk.word_tokenize)

In [ ]:
df.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

<h3>Suddivisione dataset in italiano e inglese</h3>

In [ ]:
def filtroLingua(df,lang):
    return df[df['lang'] == lang]

In [ ]:
dfIT = filtroLingua(df, 'it')

In [ ]:
dfEN = filtroLingua(df, 'en')

In [ ]:
dfIT.shape #verifico la dimensione

In [ ]:
dfEN.shape #verifico la dimensione

<h3>Rimozione stopwords</h3>

In [ ]:
from nltk.corpus import stopwords

lang = input("choose a lang: ")
stop_words = set(stopwords.words(lang))  # Import stopwords for the chosen language

def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

In [ ]:
dfEN.loc[:, 'tweetText'] = dfEN['tweetText'].apply(lambda x: cleaning_stopwords(x))

In [ ]:
dfEN['tweetText'].head()

In [ ]:
def removeUrls(text):
    return re.sub('http://\S+|https://\S+', '', text)

In [ ]:
dfEN.loc[:, 'tweetText'] = dfEN['tweetText'].apply(lambda x: removeUrls(x))

<h2>WordCloud sulla colonna di testo</h2>

In [ ]:
# Join the different processed titles together.
def createLongString(data):
    long_string = ','.join(list(data['tweetText'].values))
    return long_string

In [ ]:
str_long = createLongString(dfEN)

In [ ]:
str_long

In [ ]:
# Generate the word cloud
wc = WordCloud(
    background_color='white',
    colormap='coolwarm',
    stopwords=['meta'],
    width=1000,
    height=700,
    max_words=100,  # Maximum number of words to display
    max_font_size=150,  # Maximum font size for the words
    random_state=42,  # Set a random seed for reproducibility
    contour_color='steelblue',  # Color of the word cloud contour
    contour_width=2,  # Width of the word cloud contour
).generate(str_long)

# Display the word cloud
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

<h2>Analisi sugli Hashtag (mista per italiano e inglese)</h2>

<p>Questa analisi è valida per il dataset contenente una colonna per gli hashtags</p>

In [ ]:
exploded_df = df.explode('hashtags')
hashtags_counts = exploded_df['hashtags'].value_counts()

In [ ]:
hashtags_counts

<p>Venti hashtags/coppie di hashtags più menzionati</p>

In [ ]:
top_20_hashtags = hashtags_counts.head(20)

In [ ]:
top_20_hashtags

<p>Estrazione degli hashtag dalla colonna di testo (valido per qualsiasi dataset)</p>

In [ ]:
def findHashtags(data):
    hashtags = data['tweetText'].apply(lambda x: re.findall(r'\#\w+', x)) #trova gli hashtags con espr regolare
    all_hashtags = [tag for sublist in hashtags for tag in sublist]
    counts = Counter(listaHAshtags)
    return counts

In [ ]:
coll = findHashtags(df)

In [ ]:
for count in coll: print(count)

<p><strong>Mostro hashtag più usato</strong></p>

In [ ]:
most_common_hashtag = coll.most_common(1)[0][0]
print("L'hashtag più usato nel dataset è: "+ most_common_hashtag)

<p><strong>Mostro i 20 hashtags meno usati</strong></p>

In [ ]:
def showVals(collection):
    for value, count in collection:
        print(f"Value: {value}, Count: {count}")

In [ ]:
least_common_values = coll.most_common()[:-21:-1]
showVals(least_common_values)

<p><strong>Mostro i 20 hashtags più usati</strong></p>

In [ ]:
top_values = coll.most_common(20)

In [ ]:
showVals(top_values)

<h2>Generazione dei bigrammi</h2>

<p><i>Attenzione, usare il filtro lingua</i></p>

In [ ]:
from nltk import ngrams

In [ ]:
# Function to generate bigrams from a text
def generate_bigrams(text):
    tokens = nltk.word_tokenize(text)
    bigrams = list(ngrams(tokens, 2))
    return bigrams

# Apply the function to create a new column 'bigrams'
def applyBigrams(dfLINGUA):
    dfLINGUA['bigrams'] = dfLINGUA['tweetText'].apply(generate_bigrams)
    return dfLINGUA

In [ ]:
dfEN = applyBigrams(dfEN)

In [ ]:
dfEN.head(20)

<h2>Progressione degli hashtags nel tempo</h2>

In [ ]:
df['date'] = pd.to_datetime(df['date'], utc=True)

In [ ]:
print(df.dtypes)

In [ ]:
df['Count'] = 1

In [ ]:
grouped = df.groupby([df['date'].dt.date, 'hashtags']).sum().reset_index()

In [ ]:
pivot_df = grouped.pivot(index='date', columns='hashtags', values='Count').fillna(0)

In [ ]:
# Plot the progression of hashtags over time using a line plot
pivot_df.plot(kind='line', figsize=(10, 6))
plt.xlabel('Time', fontname = 'Arial')
plt.ylabel('Count', fontname = 'Arial')
plt.title('Hashtag Progression Over Time', fontname = 'Arial')
plt.show()